# 01 - Data Preparation

> Scaffold notebook — fill with data and code.

## Setup

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path("../data/raw")

## Load data

In [ ]:
# örnek: Euro 2020 event klasörü
events_path = list((DATA_DIR / "events").glob("*.json"))

print("Toplam event dosyası:", len(events_path))
events_path[:5]




## Clean & normalize

In [ ]:
import json

first_match_file = events_path[0]
with open(first_match_file, "r", encoding="utf-8") as f:
    match_events = json.load(f)

len(match_events), match_events[0].keys()

# event listesi → DataFrame
df = pd.json_normalize(match_events)
df.head()

shots = df[df["type.name"] == "Shot"].copy()
print("Toplam şut:", len(shots))
shots.head()

shots = shots[
    [
        "team.name",
        "player.name",
        "under_pressure",
        "shot.body_part.name",
        "shot.outcome.name",
        "shot.statsbomb_xg",   # StatsBomb xG
        "location"              # [x,y] koordinatı
    ]
]
shots.head()

# location -> shot_location_x, shot_location_y
shots["x"] = shots["location"].apply(lambda x: x[0] if isinstance(x, list) else np.nan)
shots["y"] = shots["location"].apply(lambda x: x[1] if isinstance(x, list) else np.nan)

shots.head()


processed_dir = Path("../data/processed")
processed_dir.mkdir(exist_ok=True, parents=True)

shots.to_csv(processed_dir / "shots.csv", index=False)
print("Kaydedildi:", processed_dir / "shots.csv")


## Train/val split

In [ ]:
# TODO